In [1]:
import pandas as pd 
import numpy as np 
import ast
import os
import re
import sys
import ast
import glob
import itertools

In [2]:
graph=pd.read_csv('../data/graph.tsv', sep='\t')
graph=graph.loc[:, ~graph.columns.str.contains('^Unnamed')]
graph

,key,values
0,nm0000001,"{'nm0949130': 2.5, 'nm0001677': 2.2, 'nm010356..."
1,nm0000002,"{'nm0150462': 2.0999999999999996, 'nm0000007':..."
2,nm0000003,"{'nm0131001': 4.3, 'nm0894677': 4.3, 'nm018135..."
3,nm0000004,"{'nm0161443': 3.2, 'nm0000101': 2.099999999999..."
4,nm0000005,"{'nm0090956': 2.7, 'nm0433512': 2.7, 'nm045803..."
...,...,...
1131849,nm9993613,"{'nm10720286': 4.0, 'nm5063862': 4.0, 'nm35495..."
1131850,nm9993616,"{'nm1391663': 3.5, 'nm7444538': 3.5, 'nm049125..."
1131851,nm9993650,"{'nm7144368': 4.8, 'nm2066212': 4.8, 'nm069022..."
1131852,nm9993690,"{'nm7815539': 3.5, 'nm1666585': 3.5, 'nm999369..."


In [ ]:
graph['values']=graph['values'].apply(lambda x: ast.literal_eval(x))
graph['values']=graph['values'].apply(lambda x: list(x.items()))
graph

In [ ]:
new_graph=graph.explode('values')
new_graph

In [ ]:
#Multi-indexing 
new_graph[["to_node", "value"]]= pd.DataFrame(new_graph["values"].values.tolist(), index=new_graph.index)
new_graph.drop(columns=["values"], inplace=True)
new_graph.set_index(keys=["key", "to_node"], drop= True, inplace=True)

In [ ]:
new_graph

In [ ]:
new_graph.loc['nm0762263','nm0544987']

In [ ]:
path = r'../data' # use your path
all_files = glob.glob(path + "/*_actor3.tsv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, sep="\t", header=0)
    li.append(df)

In [ ]:
IDs=[]
for i in range(len(li)):
    ids=[]
    for j in range(len(li[i])):
        ids.append((li[i].iloc[j])[0])
    IDs.append(ids)
IDs

In [ ]:
def relation_score(a,b,new_graph):
    
    #Checking if a and b have acted together
    d=(new_graph.loc[a]).to_dict()
    to_node=[]
    for k,v in d.items():
        for l,m in v.items():
            to_node.append(l)

    if b in to_node:
        #This is the starting step
        step = {
            "node": a,
            "value": 0,
            "distance": 0,
            "values": [],
            "cost": 0,
            "prev": ""
        }

        # This is all of the nodes we already found the shortest path to
        finished = {}
        # This is the next steps to consider
        queue = [step]
        # This is basically Dijkstra's shortest path algorithm
        while len(queue) > 0:
            # Take the node with the lowest cost
            queue.sort(key=lambda x: x["cost"], reverse=True)
            current = queue.pop()
            # When you take a node from the queue it means that you have found the shortest path to that node.
            finished[current["node"]] = current
            #print("Looking at " + current["node"] + ", distance: " + str(current["distance"]) + ", cost: " + str(current["cost"]))
            # Stop when we find b
            if current["node"] == b:
                #print("Found path to {} :D".format(b))
                break
            # The distance to the next nodes is 1 more than the distance to this node
            new_dist = current["distance"] + 1
            # Values is a list of the graph values so that we can calculate the average when we are calculating the cost
            new_values = current["values"]
            # Find all of the edges from this node to it's neighbours as dict
            d=(new_graph.loc[a]).to_dict()
            for k,v in d.items():
                edges=v   
            for edge in edges:
                # If the edge is in finished it means that we already found a shorter path to that node
                if finished.get(edge): continue
                val = edges[edge]
                vals = new_values + [val]
                # Our scores (genre_score, similarity) are between 0 and 10. So we want this score to be in that range aswell.
                # The distance (11 - new_dist) means that shorter paths are preferred, A->B->C=9 instead of A->E->F->C=8.
                # (10 - sum(vals)/new_dist) is 10 - avg of the scores from the graph. The graph values are the opposite
                # of the rating, so a rating of 7 => 3, 9.5 => 0.5. This is done because we want to find the shortest path between actors.
                # If we didn't do this then the path would always try to visit every actor before finding christian. This is why we
                # subtract the average of the two numbers from 10. 
                cost = 10 - (11 - new_dist + 10 - sum(vals)/new_dist) / 2

                new_step = {
                    "node": edge,
                    "value": val,
                    "distance": new_dist,
                    "values": vals,
                    "cost": cost,
                    "prev": current["node"]
                }

                in_queue = False
                # Check if it is already in the queue, if it is and this cost is lower it should be updated
                for i in range(len(queue)):
                    if queue[i]["node"] == edge:
                        if queue[i]["cost"] > new_step["cost"]:
                            queue[i] = new_step
                        in_queue = True
                        break
                if in_queue: continue
                # If the edge was not in the queue it is added.
                queue.append(new_step)


        actor = b
        path = []
        while actor != "":
            path.append(actor)
            actor = finished[actor]["prev"]
        path = list(reversed(path))
        score=(10 - finished[b]["cost"])
        #print("Leonardo-Christian score: " + str(10 - finished[christian]["cost"]))
    
    else:
        score=0

    return(score)

In [ ]:
Id_list = list(itertools.product(*IDs))
Id_list

In [ ]:
data=pd.DataFrame(Id_list)
data

In [ ]:
data.iloc[i]

In [ ]:
%%time
score=[]
for i in range(len(data)):
    x=list(itertools.combinations(list(data.iloc[i]),2))
    s=0
    for j in range(len(x)):
        s=s+relation_score(x[j][0],x[j][1],new_graph)
        if s=0:
            break
        else:
            continue
    score.append(s/len(x))

score

In [ ]:
%%time
score=[]
for i in range(len(data)):
    x=list(itertools.combinations(list(data.iloc[i]),2))
    s=0
    for j in range(len(x)):
        s=s+relation_score(x[j][0],x[j][1],new_graph)
    score.append(s/len(x))

score

In [ ]:
data['r_score']=score
data

In [ ]:
#Removing all zero scored values
data=data.loc[(data['r_score']!=0)]
#Sorting 
data=data.sort_values(by='r_score', ascending=False)
data

In [ ]:
df=data
for i in range(len(li)):
    df=pd.merge(df,li[i], how='inner', left_on=i, right_on='Actor ID')
    sname=('score_{}'.format(i))
    df[sname]=df['score']
    df=df.drop(['Actor ID', 'Avg Genre Score', 'summary_score', 'score'], axis=1)
df

In [ ]:
sname=[]
for i in range(len(li)):
    sname.append(('score_{}'.format(i)))
    
col = df.loc[: , sname]
df['gs_score'] = col.mean(axis=1)
df

In [ ]:
df['final'] = df[["r_score","gs_score"]].mean(axis=1)
col_list=[i for i in range(len(li))]
col_list.append('final')
final_df=df[col_list]
final_df=final_df.sort_values(by='final', ascending=False)
final_df

In [ ]:
%%time
#All of the {}_actor.tsv with each other 
Id1=[]
Id2=[]
score=[]
for i in range(len(IDs)):
    for j in range(len(IDs[i])):
        for m in range(i+1,len(IDs)):
            if (i+1)<(len(IDs)):
                for k in range(len(IDs[m])):
                    a=IDs[i][j]
                    b=IDs[m][k]
                    Id1.append(a)
                    Id2.append(b)
                    score.append(relation_score(a,b,new_graph))
                    #Id.append(dic)


In [ ]:
ar=[Id1,Id2]

In [ ]:
tup = list(zip(*ar))

In [ ]:
index = pd.MultiIndex.from_tuples(tup, names=['ID1', 'ID2'])

In [ ]:
rel=pd.DataFrame({'r_score':score},index=index)
rel

In [ ]:
#Removing all zero scored values
r=rel.loc[(rel['r_score']!=0)]
#Sorting 
r=r.sort_values(by='r_score', ascending=False)
r

In [ ]:
r.loc['nm0695177']

In [ ]:
pd.merge(li[0],li[1], how='outer', left_on='Actor ID', right_on='Actor ID')

In [ ]:
len(r.index.get_level_values(1).unique())

In [ ]:
len(r.index.get_level_values(0).unique())

In [ ]:
r.index.get_level_values(1).unique()

In [ ]:
r.index.get_level_values(0).unique()

In [ ]:
r=pd.DataFrame({'ID_1':Id1,'ID_2': Id2,'r_score':score}).sort_values(by='r_score', ascending=False)
r

In [ ]:
#Removing all zero scored values
re=r.loc[(r['r_score']!=0)]
re

In [ ]:
re.groupby(['ID_1','r_score'])['ID_2'].apply(list).reset_index()

In [ ]:
df1=pd.merge(li[0],re, how='inner', left_on='Actor ID', right_on='ID_1')
df1

In [ ]:
dfx=df1.drop(['ID_1'], axis=1)
dfx

In [ ]:
dfx['ID_1']=dfx['ID_2']
dfx=dfx.drop(['ID_2'],axis=1)
dfx

In [ ]:
dfy = pd.merge(dfx,re)
dfy

In [ ]:
df1=df1.groupby(['Actor ID','score_x','score_y'])['ID_2'].apply(list).reset_index()
df1

In [ ]:
df1['ID_2']

In [ ]:
li[0]

In [ ]:
li[1]

In [ ]:
li[2]